In [0]:
%pip install geopandas==0.14.3 contextily==1.6.0 pyproj==3.6.1 shapely==2.0.3

In [0]:
result = spark.sql("""
    select concat_ws('\n', collect_list(to_json(struct(*)))) as json_rows 
    from gk_los_angeles.default.events
    where order_id = '6ca8da6af1c04f82bd394fc5ff43b452' 
    and (event_type like "driver_%" or event_type = "delivered");
""").collect()[0]['json_rows']

In [0]:
result

In [0]:
import json
import geopandas as gpd
from shapely.geometry import LineString, Point
import contextily as cx
import matplotlib.pyplot as plt

# ─── your JSON-lines dump goes in `events_json` ──────────────────────────
events_json = result
# ─────────────────────────────────────────────────────────────────────

# 1️⃣ parse events & decode JSON-in-JSON
events=[]
for line in events_json.strip().splitlines():
    outer=json.loads(line)
    outer["body"]=json.loads(outer["body"])   # decode nested JSON string
    events.append(outer)

# 2️⃣ group by order
by_order={}
for ev in events:
    by_order.setdefault(ev["order_id"], []).append(ev)

# 3️⃣ construct LineStrings
GK_LAT,GK_LON = 33.9136,-118.3820   # ghost-kitchen coords
geoms,labels=[],[]
for oid,evs in by_order.items():
    poly=None
    for e in evs:
        if e["event_type"]=="driver_picked_up":
            poly=e["body"]["route_points"]; break
    if poly:
        pts=[(lon,lat) for lat,lon in poly]
    else:
        dest=next((e for e in evs if e["event_type"]=="delivered"),None)
        if not dest: continue
        pts=[(GK_LON,GK_LAT),(dest["body"]["delivered_lon"],dest["body"]["delivered_lat"])]
    geoms.append(LineString(pts)); labels.append(oid)

gdf=gpd.GeoDataFrame({"oid":labels,"geometry":geoms},crs="EPSG:4326")

# 4️⃣ plot on OSM basemap
ax=gdf.to_crs(epsg=3857).plot(figsize=(8,6),linewidth=3,alpha=.8)
gpd.GeoSeries([Point(GK_LON,GK_LAT)],crs="EPSG:4326").to_crs(epsg=3857)\
  .plot(ax=ax,marker="s",color="black",markersize=50,label="Ghost Kitchen")
cx.add_basemap(ax,source=cx.providers.OpenStreetMap.Mapnik,zoom=13)
ax.set_axis_off(); ax.set_title("Ghost-Kitchen Delivery Paths")
plt.show()
